In [98]:
import pandas as pd
import numpy as np
import copy
import itertools
# Package for Levenstein string distance
import jellyfish
import time

In [34]:
path = "C://Users//Dimitri//Desktop//ENSAE3A//NetworkData//Data//"

In [35]:
attrs = pd.read_csv(path + "attrs_sub.csv")

In [50]:
search = database["author"].str.contains("Smidt")
search2 = database["title"].str.contains("Policy")

In [51]:
ind = search[search == True].index
ind2 = search2[search2 == True].index
inter = ind.intersection(ind2).shape

In [36]:
def authors_parser(authors_string, sep=";"):
    return authors_string.split("; ")

In [104]:
def normalized_edit_distance(str1, str2):
    dist = jellyfish.levenshtein_distance(str1, str2)
    return dist / max(len(str1), len(str2))

In [165]:
def parse_names(author):
    if "," in author:
        splitted = author.split(", ")
        lastname = splitted[0]
        if len(splitted) == 1:
            firstname = ""
        else : 
            firstname = splitted[1]
            if firstname[0] == " ":
                firstname = firstname[1:]
    else:
        splitted = author.split(" ")
        if len(splitted) > 1:
            lastname = ""
            for part in splitted[1:]:
                lastname += " " + part
            firstname = splitted[0]
        else:
            firstname = ""
            lastname = splitted[0]
    return firstname, lastname

In [166]:
def sortby_firstname_len(author):
    firstname, lastname = parse_names(author)
    return len(firstname)
   

In [170]:
# Fill all authors
authors = []
for author in attrs["authors_list"]:
    authors += author
# Remove duplicates
authors = list(set(authors))

In [182]:
# Sortby first name length
authors_red = authors[0:1000]
start = time.clock()
#authors.sort(key=sortby_firstname_len, reverse=True)
authors_red.sort(key=sortby_firstname_len, reverse=True)
end = time.clock()
print(end - start)

0.0020027061946166214


In [180]:
authors

['Lopes, Francesco Saverio Stentella',
 'Hidalgo, Fernando Perez de Gracia',
 'Werlang, Sergio Ribeiro da Costa',
 'Pería, María Soledad Martínez',
 'Pería, Maria Soledad Martínez',
 'Knarvik, Karen Helene Midelfart',
 'Paul, Catherine J. Morrison',
 'Ramamurtie, Buddhavarapu Sailesh',
 'Aïssa, Mohamed Safouane Ben',
 'Jeanneney, Sylviane Guillaumont',
 'Nikitopoulos, Christina Sklibosios',
 'Martins, Marco Antonia Campos',
 'Chen, Zhuoqiong (Charlie)',
 'de Hinestrosa, Patricia Armendariz',
 'Ferreira, Rodolphe Dos Santos',
 'Chowdhury, Mohammad Tarequl H.',
 'Rodgers, Yana van der Meulen',
 'Nielsen, Lasse Holbøll Westh',
 'Oh, Frederick Dongchuhl',
 'He, Zhongzhi (Lawrence)',
 'Goldberg, Pinelopi Koujianou',
 'Pedersen, Rasmus Søndergaard',
 'Unger, Roberto Mangabeira',
 'Waller, C.J.Christopher J.',
 'de Souza, Maria Christina C.',
 'De Aghion, Beatriz Armendariz',
 'Johan, Sofia Atiqah binti',
 'Johnston, Elizabeth Tashjian',
 'Philippe-Emmanuel. Petalas',
 'Filho, Irineu de Carva

In [181]:
start = time.clock()
thresh = 0.25
authors_frame = pd.DataFrame(columns=["first_name", "last_name", "similar"])
for author in authors:
    firstname, lastname = parse_names(author)
    search = authors_frame["last_name"].str.contains(lastname)
    ind_search = search[search == True].index
    exists_similar = False
    for ind in ind_search :
        dist = normalized_edit_distance(authors_frame["first_name"][ind], 
                                        firstname)
        dist_test = dist <= thresh
        if dist_test :
            exists_similar = True
            authors_frame["similar"][ind].append(author)    
        first_letter_test1 = (len(firstname) == 1) and (firstname[0] == authors_frame["first_name"][ind][0])
        if first_letter_test1:
            exists_similar = True
            authors_frame["similar"][ind].append(author)     
    if not exists_similar:
        df = pd.DataFrame(columns=["first_name", "last_name", "similar"], index=[0])
        df.set_value(0, "first_name", firstname)
        df.set_value(0, "last_name", lastname)
        df.set_value(0, "similar", [author])
        authors_frame = authors_frame.append(df, ignore_index=True)
end = time.clock()
print(end - start)

C:\Users\Dimitri\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


KeyboardInterrupt: 

In [176]:
inds = []
for b in authors_frame.iterrows():
    if len(b[1]["similar"]) > 1:
        inds.append(b[0])

In [177]:
inds

[235, 303, 305, 449, 515]

In [188]:
df_authors = pd.DataFrame(authors, columns=["original"])

In [189]:
df_authors

,original
0,"Lopes, Francesco Saverio Stentella"
1,"Hidalgo, Fernando Perez de Gracia"
2,"Werlang, Sergio Ribeiro da Costa"
3,"Pería, María Soledad Martínez"
4,"Pería, Maria Soledad Martínez"
5,"Knarvik, Karen Helene Midelfart"
6,"Paul, Catherine J. Morrison"
7,"Ramamurtie, Buddhavarapu Sailesh"
8,"Aïssa, Mohamed Safouane Ben"
9,"Jeanneney, Sylviane Guillaumont"


In [201]:
def uniformize_names(str1):
    if "," in str1:
        splitted = str1.split(", ")
        if len(splitted) > 1:
            str1bis = splitted[1] + " " + splitted[0]
        else :
            str1bis = str1
        return str1bis
    else:
        return str1

In [202]:
df_authors["uniformat"] = df_authors["original"].apply(uniformize_names)

In [203]:
df_authors

,original,uniformat
0,"Lopes, Francesco Saverio Stentella",Francesco Saverio Stentella Lopes
1,"Hidalgo, Fernando Perez de Gracia",Fernando Perez de Gracia Hidalgo
2,"Werlang, Sergio Ribeiro da Costa",Sergio Ribeiro da Costa Werlang
3,"Pería, María Soledad Martínez",María Soledad Martínez Pería
4,"Pería, Maria Soledad Martínez",Maria Soledad Martínez Pería
5,"Knarvik, Karen Helene Midelfart",Karen Helene Midelfart Knarvik
6,"Paul, Catherine J. Morrison",Catherine J. Morrison Paul
7,"Ramamurtie, Buddhavarapu Sailesh",Buddhavarapu Sailesh Ramamurtie
8,"Aïssa, Mohamed Safouane Ben",Mohamed Safouane Ben Aïssa
9,"Jeanneney, Sylviane Guillaumont",Sylviane Guillaumont Jeanneney


In [204]:
def edit_apply(str1, str2):
    return normalized_edit_distance(str1, str2)

In [205]:
def edit_score_tab(auth, auth_df):
    tab = pd.DataFrame(data=auth_df["uniformat"].copy(), columns=["uniformat"])
    tab["edit"] = tab["uniformat"].apply(lambda x: edit_apply(auth, x))
    return tab

In [206]:
test = edit_score_tab("Francesco Saverio Stentella Lopes", df_authors)

In [207]:
test

,uniformat,edit
0,Francesco Saverio Stentella Lopes,0.000000
1,Fernando Perez de Gracia Hidalgo,0.787879
2,Sergio Ribeiro da Costa Werlang,0.848485
3,María Soledad Martínez Pería,0.787879
4,Maria Soledad Martínez Pería,0.787879
5,Karen Helene Midelfart Knarvik,0.848485
6,Catherine J. Morrison Paul,0.848485
7,Buddhavarapu Sailesh Ramamurtie,0.909091
8,Mohamed Safouane Ben Aïssa,0.757576
9,Sylviane Guillaumont Jeanneney,0.848485


In [250]:
df_authors_reduced = df_authors.iloc[0:1000, :].copy()

In [251]:
def map_authors(auths_df, thresh):
    auths_df["equivalent"] = None
    for i in auths_df.index:
        auth = auths_df.get_value(i, "uniformat")
        auth_original = auths_df.get_value(i, "original")
        auths_df.set_value(i, "equivalent", [auth_original])
        search = auths_df.drop(i, axis=0)["uniformat"].str.contains(auth.split(" ")[-1])
        ind_search = search[search == True].index
        if len(ind_search) >= 1:
            sub_auths_df = auths_df.iloc[ind_search, :]
            sub_auths_df["edit"] = sub_auths_df["uniformat"].apply(lambda x: edit_apply(auth, x))
            sub_inds = sub_auths_df[sub_auths_df["edit"] <= thresh].index
            if len(sub_inds) > 0:
                for j in sub_inds:
                    auths_df.loc[i, "equivalent"].append(auths_df.get_value(j, "original"))
    return auths_df
            

In [ ]:
start = time.clock()
dd = map_authors(df_authors_reduced, 0.25)
end = time.clock()
print(end - start)

C:\Users\Dimitri\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [243]:
dd

,original,uniformat,equivalent
0,"Lopes, Francesco Saverio Stentella",Francesco Saverio Stentella Lopes,"[Lopes, Francesco Saverio Stentella, Stentella..."
1,"Hidalgo, Fernando Perez de Gracia",Fernando Perez de Gracia Hidalgo,"[Hidalgo, Fernando Perez de Gracia]"
2,"Werlang, Sergio Ribeiro da Costa",Sergio Ribeiro da Costa Werlang,"[Werlang, Sergio Ribeiro da Costa, da Costa We..."
3,"Pería, María Soledad Martínez",María Soledad Martínez Pería,"[Pería, María Soledad Martínez, Pería, Maria S..."
4,"Pería, Maria Soledad Martínez",Maria Soledad Martínez Pería,"[Pería, Maria Soledad Martínez, Pería, María S..."
5,"Knarvik, Karen Helene Midelfart",Karen Helene Midelfart Knarvik,"[Knarvik, Karen Helene Midelfart, Midelfart Kn..."
6,"Paul, Catherine J. Morrison",Catherine J. Morrison Paul,"[Paul, Catherine J. Morrison]"
7,"Ramamurtie, Buddhavarapu Sailesh",Buddhavarapu Sailesh Ramamurtie,"[Ramamurtie, Buddhavarapu Sailesh]"
8,"Aïssa, Mohamed Safouane Ben",Mohamed Safouane Ben Aïssa,"[Aïssa, Mohamed Safouane Ben]"
9,"Jeanneney, Sylviane Guillaumont",Sylviane Guillaumont Jeanneney,"[Jeanneney, Sylviane Guillaumont]"


In [219]:
dd["equivalent"][1][0] == dd["equivalent"][1][1]

True